# 1.데이터 수집- OpenAPI 사용하기

In [ ]:
import requests
import pandas as pd

## 2. 검색 결과 페이지 가져오기 - "20대가 가장 좋아하는 도서목록"

In [ ]:
!pip install gdown

In [ ]:
import gdown
gdown.download("https://bit.ly/3q9SZix", "examples/20s_best_book.json")

In [ ]:
# json file 로딩 -> dataFrame 객체 변환
books_df = pd.read_json('examples/20s_best_book.json')
books_df.head()

In [ ]:
# 특정 열을 선택하여 새 데이터프레임 생성 - 컬럼 인덱싱 이용
books = books_df[['no','ranking','bookname','authors','publisher',
                 'publication_year','isbn13']]
books.head()

In [ ]:
# 원하는 데이터프레임 행과 열 동시에 선택하기 - df.loc[]
books_df.loc[[0,1], ['bookname','authors']]

In [ ]:
books = books_df.loc[:, 'no':'isbn13']
books.head()

In [ ]:
books_df.loc[::2, 'no':'isbn13'].head()

## 3. 검색 결과 HTML 페이지 가져오기 - requests.get()

In [ ]:
import requests

isbn = 9791190090018
url = 'https://www.yes24.com/product/search?query=isbn'
r = requests.get(url)

# print(r.text)

## 4. HTML에서 데이터 추출하기 - BeautifulSoup
    1. 크롬 브라우저 개발자 도구로 HTML 태그 찾기
    2. 태그 위치 찾기 : soup.find('찾을 태그 이름',attrs={'태그 속성을 딕셔너리로 지정'} 
    

### [문제] BeautifulSoup으로 HTML에서 첫 번째 도서명의 링크 주소 찾기

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
prd_link = soup.find('a', attrs={'class':'gd_name'})
print(prd_link)

In [ ]:
print(prd_link['href'])

### [문제] BeautifulSoup으로 HTML에서 첫 번째 도서의 페이지수가 담긴 HTML의 위치 찾기

In [ ]:
url = 'https://www.yes24.com/Product/Goods/74261416'
r= requests.get(url)

In [ ]:
# soup.select()
soup = BeautifulSoup(r.text, 'html.parser')
prd_detail= soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td')
print(prd_detail)

for result  in prd_detail:
    print(result.text)

page = result.text.split('|')[0]
print(page)

In [ ]:
# soup.find_all()
soup = BeautifulSoup(r.text, 'html.parser')
prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
print(prd_detail)
prd_tr_list = prd_detail.find_all('tr')
print(prd_tr_list)

In [ ]:
for tr in prd_tr_list:
    if tr.find('th').get_text() == '쪽수, 무게, 크기':
        page_td = tr.find('td').get_text()
        break

In [ ]:
print(page_td)

In [ ]:
print(page_td.split()[0])

### [문제]  20대가 선호하는 전체 도서의 쪽수 구하기

In [ ]:
#1. 사용자 정의 함수
def get_page_cnt(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[0]
    return ''

In [ ]:
get_page_cnt(9791190090018)

In [ ]:
top10_books = books.head(10) # 인기 도서 10권 추출
top10_books

In [ ]:
# 10개 도서의 쪽수를 한번에 구하기
def get_page_cnt2(row):
    isbn = row['isbn13']
    return get_page_cnt(isbn)

In [ ]:
# 각 행의 반복 작업을 수행하기 위해 데이터프레임의 apply()메서드 적용
page_count = top10_books.apply(get_page_cnt2, axis = 1)
print(page_count)

In [ ]:
# 새로운 시리즈 생성
page_count = top10_books.apply(lambda row: get_page_cnt(row['isbn13']),axis =1)
page_count

In [ ]:
page_count.name = 'page_count'
print(page_count)

### [문제] 데이터프레임과 시리즈 합치기 : merge()

In [ ]:
top10_with_page_count = pd.merge(top10_books, page_count,
                                 left_index=True, right_index=True)
top10_with_page_count

## `merge()` 함수의 매개변수

In [ ]:
df1 = pd.DataFrame({'col1': ['a','b','c'], 'col2': [1,2,3]})
df1

In [ ]:
df2 = pd.DataFrame({'col1': ['a','b','d'], 'col3': [10,20,30]})
df2

In [ ]:
#df1.merge(df2, on = 'col1')

In [ ]:
pd.merge(df1, df2, on='col1')


In [ ]:
pd.merge(df1, df2, how='left', on='col1')

In [ ]:
pd.merge(df1, df2, how='right', on='col1')

In [ ]:
pd.merge(df1, df2, how='outer', on='col1')

In [ ]:
pd.merge(df1, df2, left_on='col1', right_on='col1')

In [ ]:
pd.merge(df1, df2, left_on='col2', right_index=True)